# Tensor‑Train SVD (TT‑SVD)

In [21]:
import numpy as np

In [22]:

def choose_rank(s, eps):
    for k in range(1, len(s)):
        if np.sum(s[k:] ** 2) <= eps:
            return k
    return len(s)


In [23]:

def tt_svd(tensor, eps=1e-8):
    dims = tensor.shape
    d = len(dims)
    factors = []
    core = tensor.reshape(dims[0], -1)

    for i in range(d - 1):
        U, s, Vh = np.linalg.svd(core, full_matrices=False)
        k = choose_rank(s, eps)
        U_k, s_k, Vh_k = U[:, :k], s[:k], Vh[:k, :]

        if i == 0:
            factors.append(U_k)
        else:
            r_prev = factors[-1].shape[-1]
            factors.append(U_k.reshape(r_prev, dims[i], k)) 

        core = s_k[:, None] * Vh_k 
        if i < d - 2:
            core = core.reshape(k * dims[i + 1], -1)
        else:
            factors.append(core.reshape(k, dims[i + 1]))

    return factors


In [24]:

def tt_to_tensor(factors):
    res = factors[0]  
    for G in factors[1:-1]:
        res = np.tensordot(res, G, axes=([-1], [0]))
    res = np.tensordot(res, factors[-1], axes=([-1], [0])) 
    return res


In [ ]:

sizes = (10, 15, 20)
T = np.random.randn(*sizes)

factors = tt_svd(T, eps=1e-10)
T_rec = tt_to_tensor(factors)

print("ошибка:", np.linalg.norm(T_rec - T) / np.linalg.norm(T))
for i, f in enumerate(factors):
    print(f"G{i+1} shape: {f.shape}")


Относительная ошибка: 2.3589929569558555e-15
G1 shape: (10, 10)
G2 shape: (10, 15, 20)
G3 shape: (20, 20)
